In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from sklearn.datasets.california_housing import fetch_california_housing
housing=fetch_california_housing()
# print(housing.head())

In [ ]:
type(housing)

In [ ]:
housing.DESCR


In [ ]:
housing.data.shape

In [ ]:
housing.data[0]

In [ ]:
from sklearn import tree
dtr=tree.DecisionTreeRegressor(max_depth=2)
dtr.fit(housing.data[:,[6,7]],housing.target)

In [ ]:
print('1'*10)
print(dtr)
print('2'*10)


In [ ]:
housing.target

In [ ]:
housing.feature_names

In [ ]:
dot_data=tree.export_graphviz(
    dtr,
    out_file=None,
    feature_names=housing.feature_names[6:8],
    filled=True,
    impurity=False,
    rounded=True
)

In [ ]:
housing.feature_names[6:8]

In [ ]:
# pip install pydotplus 
import pydotplus
graph=pydotplus.graph_from_dot_data(dot_data)
graph.get_nodes()[7].set_fillcolor('#FFF2DD')

In [ ]:
type(dot_data)

In [ ]:
from IPython.display import Image
Image(graph.create_png())

In [ ]:
graph.write_png('dtr_white_background.png')

In [ ]:
from sklearn.model_selection import train_test_split
data_train,data_test,target_train,target_test=train_test_split(
    housing.data,housing.target,test_size=0.1,random_state=42 )
dtr=tree.DecisionTreeRegressor(random_state=42)
dtr.fit(data_train,target_train)
dtr.score(data_test,target_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(random_state=42)
rfr.fit(data_train,target_train)
rfr.score(data_test,target_test )

## 树模型参数:
1.criterion gini or entropy

2.splitter best or random 前者是在所有特征中找最好的切分点 后者是在部分特征中（数据量大的时候）

3.max_features None（所有），log2，sqrt，N 特征小于50的时候一般使用所有的

4.max_depth 数据少或者特征少的时候可以不管这个值，如果模型样本量多，特征也多的情况下，可以尝试限制

5.min_samples_split 如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。
 
6.min_samples_leaf 这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝，如果样本量不大，不需要管这个值，大些如10W可是尝试下5
 
7.min_weight_fraction_leaf 这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

8.max_leaf_nodes 通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制具体的值可以通过交叉验证得到。

9.class_weight 指定样本各类别的的权重，主要是为了防止训练集某些类别的样本过多导致训练的决策树过于偏向这些类别。这里可以自己指定各个样本的权重如果使用“balanced”，则算法会自己计算权重，样本量少的类别所对应的样本权重会高。

10.min_impurity_split 这个值限制了决策树的增长，如果某节点的不纯度(基尼系数，信息增益，均方差，绝对差)小于这个阈值则该节点不再生成子节点。即为叶子节点 。

n_estimators:要建立树的个数

In [22]:
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
tree_param_grid={'min_samples_split':list((3,6,9)),'n_estimators':list((10,50,100))}
grid=GridSearchCV(RandomForestRegressor(),param_grid=tree_param_grid,cv=5)
grid.fit(data_train,target_train)
# grid.grid_scores_,grid.best_params_,grid.best_score_
grid.cv_results_,grid.best_params_,grid.best_score_

({'mean_fit_time': array([ 0.94700475,  4.81647754,  9.91325684,  0.85481114,  4.35702777,
         10.10270576,  0.98847332,  5.61431499, 10.60003657]),
  'std_fit_time': array([0.04095955, 0.29745729, 0.29947909, 0.02368189, 0.05596329,
         0.80794979, 0.07675006, 1.38417099, 0.93935008]),
  'mean_score_time': array([0.01359549, 0.05400925, 0.11279993, 0.01000447, 0.05100193,
         0.11004186, 0.01039748, 0.06000276, 0.09912863]),
  'std_score_time': array([0.00174992, 0.00282588, 0.00688157, 0.00126649, 0.00648215,
         0.0151492 , 0.00162267, 0.01691047, 0.00956282]),
  'param_min_samples_split': masked_array(data=[3, 3, 3, 6, 6, 6, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_n_estimators': masked_array(data=[10, 50, 100, 10, 50, 100, 10, 50, 100],
               mask=[False, False, False, False, False, False, False, False,
             

In [23]:
grid.cv_results_

{'mean_fit_time': array([ 0.94700475,  4.81647754,  9.91325684,  0.85481114,  4.35702777,
        10.10270576,  0.98847332,  5.61431499, 10.60003657]),
 'std_fit_time': array([0.04095955, 0.29745729, 0.29947909, 0.02368189, 0.05596329,
        0.80794979, 0.07675006, 1.38417099, 0.93935008]),
 'mean_score_time': array([0.01359549, 0.05400925, 0.11279993, 0.01000447, 0.05100193,
        0.11004186, 0.01039748, 0.06000276, 0.09912863]),
 'std_score_time': array([0.00174992, 0.00282588, 0.00688157, 0.00126649, 0.00648215,
        0.0151492 , 0.00162267, 0.01691047, 0.00956282]),
 'param_min_samples_split': masked_array(data=[3, 3, 3, 6, 6, 6, 9, 9, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 50, 100, 10, 50, 100, 10, 50, 100],
              mask=[False, False, False, False, False, False, False, False,
                    False],


In [24]:
grid.best_params_

{'min_samples_split': 6, 'n_estimators': 100}

In [25]:
grid.best_score_

0.8067083357798499

GridSearchCV（网络搜索交叉验证）用于系统地遍历模型的多种参数组合，通过交叉验证从而确定最佳参数，适用于小数据集。

常用属性 

best_score_ ：最佳模型下的分数

best_params_ ：最佳模型参数

grid_scores_ ：模型不同参数下交叉验证的平均分

cv_results_ ： 具体用法模型不同参数下交叉验证的结果

best_estimator_ : 最佳分类器之所以出现以上问题

